# Video Data EM Notebook

In [ ]:
import sys
import py_entitymatching as em
import pandas as pd
import os
import time

###################################################################
#KEY!!!! VARIABLE PREVENTS OVERWRITING LABELED SAMPLED DATA
###################################################################
GENERATE_NEW_LABELED_DATA = False
GENERATE_NEW_TEST_TRAIN_DATA = False



Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

In [3]:
# Get the paths
path_A = '../DATA/imdb3_neg_nan.csv'
path_B = '../DATA/thenumbers3_neg_nan.csv'
print(path_A)

../DATA/imdb3_neg_nan.csv


In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
A.head()

No handlers could be found for logger "py_entitymatching.io.parsers"


,id,title,year,mpaa,runtime,genres,director,stars,gross
0,0,Toy Story 2,1999,G,92 min,"Animation, Adventure, Comedy",John Lasseter,"Ash Brannon,Lee Unkrich,Tom Hanks,Tim Allen,Joan Cusack,Kelsey Grammer",$245.85M
1,1,The Outlaw Josey Wales,1976,PG,135 min,Western,Clint Eastwood,"Clint Eastwood,Sondra Locke,Chief Dan George,Bill McKinney",$31.80M
2,2,"Monsters, Inc.",2001,G,92 min,"Animation, Adventure, Comedy",Pete Docter,"David Silverman,Lee Unkrich,Billy Crystal,John Goodman,Mary Gibbs,Steve Buscemi",$289.92M
3,3,In the Heat of the Night,1967,Not Rated,110 min,"Crime, Drama, Mystery",Norman Jewison,"Sidney Poitier,Rod Steiger,Warren Oates,Lee Grant",$24.38M
4,4,Chungking Express,1994,PG-13,102 min,"Crime, Drama, Romance",Kar-Wai Wong,"Brigitte Lin,Takeshi Kaneshiro,Tony Chiu-Wai Leung,Faye Wong",$0.60M


In [5]:
B.head()

,id,title,year,mpaa,runtime,genres,director,stars,gross
0,0,Effects,2005,-1,-1,-1,Dusty Nelson,-1,-1
1,1,Ek Haseena Thi Ek Deewana Tha,2017,-1,105 minutes,Drama,Suneel Darshan,-1,"$149,491"
2,2,Ekk Albela,2016,-1,-1,Drama,Shekhar Sartandel,-1,"$1,907"
3,3,The Daisy Chain,2009,R,-1,-1,-1,-1,-1
4,4,Dali & I: The Surreal Story,2011,-1,-1,Drama,Andrew Niccol,Al Pacino,-1


### Check for null entries

In [6]:
B.isnull().sum()

id          0
title       0
year        0
mpaa        0
runtime     0
genres      0
director    0
stars       0
gross       0
dtype: int64

In [7]:
A.isnull().sum()

id          0
title       0
year        0
mpaa        0
runtime     0
genres      0
director    0
stars       0
gross       0
dtype: int64

### Check the data size

In [8]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 4291
Number of tuples in B: 31006
Number of tuples in A X B (i.e the cartesian product): 133046746


In [9]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [10]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 200, 1, show_progress=False)
len(A1), len(B1)

# But for the purposes of this notebook, we will use the entire table A and B

(125, 200)

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables to reduce computational complexity. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. We can use a mix and match of these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two movies with different titles will not match. So we decide the apply blocking over names:

In [11]:
# Initial Blocking plan

# A, B -- overlap blocker [title] --------------------|---> candidate set

In [12]:
# Create an overlap blocker
ab = em.OverlapBlocker()

# Block using 'title' attribute
C1 = ab.block_tables(A, B, 'title', 'title', 
                    l_output_attrs=['title', 'year', 'mpaa', 'runtime', 'genres', 'director', 'stars', 'gross'], 
                    r_output_attrs=['title', 'year', 'mpaa', 'runtime', 'genres', 'director', 'stars', 'gross'],
                    overlap_size=1
                    )

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:38


In [13]:
len(C1)

10956134

## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10,956,134 (from 133,046,746, approx. 8%), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [14]:
# Debug blocker output
startTime = time.time()
dbg = em.debug_blocker(C1, A, B, output_size=200, attr_corres=[('title','title'), ('year', 'year')])
endTime = time.time()
print("Total time: %.2f seconds."%(endTime-startTime))

Total time: 44.95 seconds.


In [15]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head(50)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,rtable_title,rtable_year
0,0,2965,17445,Advise & Consent,1962,Ginger & Rosa,2013
1,1,2965,17842,Advise & Consent,1962,Harry & Son,1984
2,2,2965,17844,Advise & Consent,1962,Harry & Snowman,2016
3,3,2965,19318,Advise & Consent,1962,Lambert & Stamp,2015
4,4,2965,19600,Advise & Consent,1962,David & Layla,2007
5,5,2965,19796,Advise & Consent,1962,Hansel & Gretel,2002
6,6,2965,20161,Advise & Consent,1962,Q & A,1990
7,7,2965,23293,Advise & Consent,1962,Starsky & Hutch,2004
8,8,2965,23439,Advise & Consent,1962,Spring & Arnaud,2016
9,9,2965,25285,Advise & Consent,1962,Town & Country,2001


From the debug blocker's output we observe that the current blocker retains quite a lot of obvious mismatches. We would want to update the blocking sequence to drop few more of these obvious mismatches.

For the considered dataset, we know that for the movies to match the year of release must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to find the intersection of the outputs from the two overlap blockers to get a consolidated candidate set.

In [ ]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--

In [ ]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'year' attribute 
C2 = ob.block_candset(C1, 'year', 'year' 
                    )
len(C2)

0% [##################            ] 100% | ETA: 00:01:22

In [ ]:
# Display first two rows from C2
C2.head()

We similarily reduce number of potential tuple pairs by matching mpaa ratings of the movies as well.

In [ ]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--
#
# C2 ------ overlap blocker [mpaa] --------> C3--

In [ ]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C3 = ob.block_candset(C2, 'mpaa', 'mpaa' 
                    )
len(C3)

In [ ]:
# Display first two rows from C3
C3.head()

We observe that a lot of potential matches in our first overlap blocker are due to the word *the* being matched between two movie titles. We wish to filter out such tuple pairs. Hence we write a customized black box blocker additionally into the pipeline.

In [ ]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--
#
# C2 ------ overlap blocker [mpaa] --------> C3--
#
# C3 ------ black box [title match more than just 'the'] --------> C4--

In [ ]:
# Create blocker to block tuples that only match based on 'the'.
def overlap_ignoring_words(ltuple, rtuple):
    words = set(['the'])
    # Remove ignore words set from strings.
    lostring = ltuple['title'].lower().split()
    rostring = rtuple['title'].lower().split()
    l_tokens = []
    r_tokens = []
    for word in lostring:
        if word.strip() not in words:
            l_tokens.append(word.strip())
    for word in rostring:
        if word.strip() not in words:
            r_tokens.append(word.strip())
    l_tokens = set(l_tokens)
    r_tokens = set(r_tokens)
        
    # Compute overlap.
    #l_tokens = set(list(map(lambda item: item.strip(), lstring.split())))
    #r_tokens = set(list(map(lambda item: item.strip(), rstring.split())))
    intersection = l_tokens.intersection(r_tokens)
    if len(intersection) >= 1:
        return False
    else:
        return True
    
# Create and apply blocker.
bb = em.BlackBoxBlocker()
bb.set_black_box_function(overlap_ignoring_words)
C4 = bb.block_candset(C3)    
    

In [ ]:
C4.head()

In [ ]:
len(C4)
#Save all tuple pairs remainng after blocking step
em.to_csv_metadata(C4, '../DATA/AllTuplePairs.csv')

We can proceed with the matching step now.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 600 tuple pairs for labeling purposes.

In [ ]:
# Sample  candidate set
S = em.sample_table(C4, 600)
S.head()

In [ ]:
if GENERATE_NEW_LABELED_DATA:
    # Label S interactively. 
    G = em.label_table(S, 'gold')
    # SAVE! TIS A BITCH TO LABEL S AGAIN, AND IT NEEDS TO BE LABELED WITHIN THE UI!
    em.to_csv_metadata(G, '../DATA/G.csv')
else:
    # Load the dataset.
    G = em.read_csv_metadata('../DATA/G.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')


## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [ ]:
if GENERATE_NEW_TEST_TRAIN_DATA:
    # Split S into development set (I) and evaluation set (J)
    IJ = em.split_train_test(G, train_proportion=0.7, random_state=5)
    I = IJ['train']
    J = IJ['test']
    print(len(I[I['gold']==1]))
    print(len(J[J['gold']==1]))
    em.to_csv_metadata(I, '../DATA/I.csv')
    em.to_csv_metadata(J, '../DATA/J.csv')
else:
    # Load the dataset.
    I = em.read_csv_metadata('../DATA/I.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
    J = em.read_csv_metadata('../DATA/J.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables.

In [ ]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [ ]:
# List the names of the features generated
feature_table['feature_name']

### Converting the development set to feature vectors

In [ ]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)

In [ ]:
# Display first few rows
H.head(3)

### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. We use five fold cross validation and use 'F1 score' metric to select the best matcher.

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']#Training set performance measure using internal CV

Clearly, the random forrest model outperforms others. 

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. The selected model is Random Forrest.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
# Predict on L 
predictions = rf.predict(table=L,#The entire evaluation set
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

### Evaluating all models

Train all models

In [ ]:
# Train using feature vectors from I 
dt.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Train using feature vectors from I 
rf.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Train using feature vectors from I 
svm.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Train using feature vectors from I 
lg.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Train using feature vectors from I 
ln.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')
# Train using feature vectors from I 
nb.fit(table=H, #the entire development set
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')




Perform predictions for every model and output results

In [ ]:
# Predict on L
models = [dt, rf, svm, ln, lg, nb]
modelNames = ['dt', 'rf', 'svm', 'ln', 'lg', 'nb']

for i, model in enumerate(models):
    predictions = model.predict(table=L,#The entire evaluation set
                         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)
    eval_result = em.eval_matches(predictions, 'gold', 'predicted')
    print(modelNames[i])
    em.print_eval_summary(eval_result)
    print('\n')
    